### LSTM Analysis 


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### 1. Data Preparation



In [6]:
voo_data = pd.read_csv('../datasets/VOO.csv')
display(voo_data)

,Date,Open,High,Low,Close,Adj Close,Volume
0,3/1/2021,354.549988,359.390015,354.500000,358.119995,341.877502,3721100
1,3/2/2021,358.380005,358.630005,355.160004,355.350006,339.233154,5462600
2,3/3/2021,354.700012,355.640015,350.559998,350.660004,334.755859,6317600
3,3/4/2021,350.489990,353.019989,341.920013,346.339996,330.631775,6604500
4,3/5/2021,349.769989,353.730011,342.589996,352.690002,336.693817,8721300
...,...,...,...,...,...,...,...
499,2/22/2023,367.179993,368.600006,364.839996,366.329987,360.530518,3872000
500,2/23/2023,369.070007,369.570007,364.130005,368.170013,362.341400,2867200
501,2/24/2023,363.309998,365.029999,361.730011,364.230011,358.463776,4055000
502,2/27/2023,367.489990,368.769989,364.589996,365.489990,359.703796,3300100
